<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/04-Large_Language_Model/04.02_Large_Language_Model_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

In this notebook you'll be using two LLMs. Google `gemini-pro` and Self - hosted in - notebook `databricks/dolly-v2`.

In [1]:
!pip -q install google-generativeai accelerate transformers sentence-transformers tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s eta 0:00:00


## Authenticate notebook to Google Cloud API
You can get your Google Cloud API key at https://console.cloud.google.com/apis/credentials

For security reason we recomment to restrict the key to allow only Generative Language API

In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key: ")

Provide your Google API Key: ··········


Test that we have access to Google APIs by requesting the list of models

In [3]:
import google.generativeai as genai

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Initialize `databricks/dolly-v2-3b` via [HuggingFace](https://huggingface.co/databricks/dolly-v2-3b). Multiple progressively more powerful models are available, including 3b, 7b and 12b (referring to Billions of parameters). `dolly-v2-3b` is the only model in the family that would fit in the memory and GPU available in a free Google Colab instance.

Loading and initializing the model can take few minutes.

In [4]:
""" # skip dolly initialization
import torch
from transformers import pipeline
dolly_completion = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto")
"""

' # skip dolly initialization\ndolly_completion = pipeline(model="databricks/dolly-v2-3b", \n                         torch_dtype=torch.bfloat16, \n                         trust_remote_code=True, \n                         device_map="auto")\n'

Helper function for Google Gemini model

In [5]:
model = genai.GenerativeModel('gemini-pro')
def gemini_completion(prompt):
    response = model.generate_content(prompt)
    return response

# Create the prompt

Prompt contains instructions, context and the question. Feel free to experiment with the prompt and see the difference in responses from different models.

News article used in this example: https://www.cnn.com/2023/05/18/media/disney-florida-desantis/index.html

In [6]:
context = """

Disney on Thursday upped the ante in its battle with Florida’s Republican Gov. Ron DeSantis, and it cost his state 2,000 white-collar jobs.
Disney is scrapping plans to build a $1 billion office complex in Florida, citing “changing business conditions,” according to a memo provided by a Disney spokesperson.
The decision comes at a time when the company is openly feuding with DeSantis, who is expected to officially enter the 2024 GOP presidential race next week, CNN reported Thursday.
A spokesperson for DeSantis said it was “unsurprising” that Disney would cancel the project “given the company’s financial straits, falling market cap and declining stock price.”
Disney, along with the broader media industry, is grappling with a difficult advertising environment and a massive writers strike. Earlier this year it announced it would be cutting 7,000 jobs as part of a cost-cutting effort.
Separately, the company confirmed Thursday that it would shut down its Star Wars: Galactic Starcruiser resort at Disney World just over a year after it opened.
The popular attraction “will take its final voyage” at the end of September, Disney said, adding that it is working with guests to rebook reservations for later in the year.
"""

question="What plans Disney is cancelling?"

prompt = f"""
Instruction: Use only information in the following context to answer the question at the end.
If you don't know, say that you do not know.

Context:  {context}

Question: {question}

Response:
"""
print(prompt)

#res = dolly_completion(prompt)
#print("Dolly:")
#print(res[0]['generated_text'])


res = gemini_completion(prompt)
print("\nGemini:")
print(res.candidates[0].content.parts[0].text)


Instruction: Use only information in the following context to answer the question at the end. 
If you don't know, say that you do not know. 
 
Context:  

Disney on Thursday upped the ante in its battle with Florida’s Republican Gov. Ron DeSantis, and it cost his state 2,000 white-collar jobs.
Disney is scrapping plans to build a $1 billion office complex in Florida, citing “changing business conditions,” according to a memo provided by a Disney spokesperson.
The decision comes at a time when the company is openly feuding with DeSantis, who is expected to officially enter the 2024 GOP presidential race next week, CNN reported Thursday.
A spokesperson for DeSantis said it was “unsurprising” that Disney would cancel the project “given the company’s financial straits, falling market cap and declining stock price.”
Disney, along with the broader media industry, is grappling with a difficult advertising environment and a massive writers strike. Earlier this year it announced it would be cu

## TODO:

Some ideas for you to try:
- add "Respond in French/Spanish" to the prompt.

-  more information into the context until you hit the token limit of the model.

- Replace the entire prompt with a simple task like "Tell me about Newmarket, Ontario"

- provoke hallucinations "what was the name of the first elephant to walk on the moon?"

In [7]:
prompt = "Tell me about Newmarket, Ontario"

#res = dolly_completion(prompt)
#print("Dolly:")
#print(res[0]['generated_text'])


res = gemini_completion(prompt)
print("\nGemini:")
print(res.candidates[0].content.parts[0].text)


Gemini:
**Newmarket, Ontario**

**Overview:**

Newmarket is a vibrant and growing town located in York Region, approximately 40 kilometers north of Toronto. With a population of approximately 87,000, Newmarket is the 12th largest urban center in the Greater Toronto Area (GTA).

**History:**

* Newmarket was founded in 1794 by a group of Quakers from Pennsylvania.
* The town was named after Newmarket in England, which was the birthplace of William Holland, one of the Quaker founders.
* Newmarket played a significant role in the War of 1812, as it was a hub for British troops and supplies.
* In the mid-19th century, Newmarket became a prosperous market town, known for its agricultural produce.
* Industrialization began to take hold in the late 19th century, with the establishment of several factories and mills.

**Demographics:**

* Newmarket has a diverse population, with a large percentage of immigrants from various countries around the world.
* The majority of residents speak English